# Violent Crime Statistics - Data Processing Steps

This Jupyter notebook documents the repeatable steps of processing the DC violent crime data by Census block group. The below worklow is summarized here:

1. Extract violent crime data from full crime dataset (from AWS to Pandas dataframe)
2. Group data by block group and unstack to see crimes by crime type
3. Complete formatting of pandas dataframe
4. Load final violent crime statistics data to AWS

Once the violent crime data and the socioeceonomic data are available via AWS, they can be remotely joined and be ready for machine learning analysis.

## 1. Load Violent Crime Data

In [1]:
# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
from config import remote_db_endpoint, remote_db_port
from config import remote_dccrime_dbname, remote_dccrime_dbuser, remote_dccrime_dbpwd

# Import Pandas
import pandas as pd

In [2]:
# Create a remote database engine connection
# AWS Database Connection
engine = create_engine(f"mysql://{remote_dccrime_dbuser}:{remote_dccrime_dbpwd}@{remote_db_endpoint}:{remote_db_port}/{remote_dccrime_dbname}")
conn = engine.connect()


In [3]:
# From AWS database, only extract violent crime incidents and only the offense, block group, and count-of-each.

violent_crime = pd.read_sql("SELECT OFFENSE, BLOCK_GROUP, COUNT(*) from crime_incidents_all WHERE OFFENSE IN ('ASSAULT W/DANGEROUS WEAPON', 'HOMICIDE', 'ROBBERY', 'SEX ABUSE') GROUP BY OFFENSE, BLOCK_GROUP", conn)
print(len(violent_crime))
violent_crime.head(5)

1432


,OFFENSE,BLOCK_GROUP,COUNT(*)
0,ASSAULT W/DANGEROUS WEAPON,,9
1,ASSAULT W/DANGEROUS WEAPON,000100 1,3
2,ASSAULT W/DANGEROUS WEAPON,000100 2,3
3,ASSAULT W/DANGEROUS WEAPON,000100 3,7
4,ASSAULT W/DANGEROUS WEAPON,000100 4,31


In [4]:
# Dataframe column formatting
violent_crime_df = violent_crime.rename(columns={"OFFENSE":"Offense", "BLOCK_GROUP":"Block Group", "COUNT(*)": "Count"})
violent_crime_df['Block Group'] = violent_crime_df['Block Group'].str.replace(" ","")
violent_crime_df.head()

,Offense,Block Group,Count
0,ASSAULT W/DANGEROUS WEAPON,,9
1,ASSAULT W/DANGEROUS WEAPON,0001001,3
2,ASSAULT W/DANGEROUS WEAPON,0001002,3
3,ASSAULT W/DANGEROUS WEAPON,0001003,7
4,ASSAULT W/DANGEROUS WEAPON,0001004,31


## 2. Group Data by Block Group and Offense

In [5]:
# Group violent crime data by block group and offense
grouped_violent = violent_crime_df.groupby(['Block Group', 'Offense'])
grouped_violent_df = grouped_violent.sum()

In [6]:
# Unstack dataframe so that offense types are listed across column headers
grouped_violent_df2 = grouped_violent_df.unstack()
grouped_violent_df2.head()

Count                           
Offense     ASSAULT W/DANGEROUS WEAPON HOMICIDE ROBBERY SEX ABUSE
Block Group                                                      
                                   9.0      2.0     4.0       2.0
0001001                            3.0      NaN     2.0       1.0
0001002                            3.0      NaN    10.0       2.0
0001003                            7.0      NaN     8.0       2.0
0001004                           31.0      NaN    36.0       7.0

## 3. Clean Dataframe Formatting

In [7]:
# Create new cleaner dataframe of crime data
ADW_sum = grouped_violent_df2.groupby(["Block Group"]).sum()["Count"]["ASSAULT W/DANGEROUS WEAPON"]
homicide_sum = grouped_violent_df2.groupby(["Block Group"]).sum()["Count"]["HOMICIDE"]
robbery_sum = grouped_violent_df2.groupby(["Block Group"]).sum()["Count"]["ROBBERY"]
sexabuse_sum = grouped_violent_df2.groupby(["Block Group"]).sum()["Count"]["SEX ABUSE"]

crime_totals = pd.DataFrame({"ADW": ADW_sum,
                                   "Homicide": homicide_sum,
                                   "Robbery": robbery_sum,
                                   "Sex Abuse": sexabuse_sum})


crime_totals.drop(crime_totals.index[0], inplace=True)

crime_totals['BG'] = crime_totals.index
crime_totals.head()

,ADW,Homicide,Robbery,Sex Abuse,BG
Block Group,,,,,
0001001,3.0,0.0,2.0,1.0,0001001
0001002,3.0,0.0,10.0,2.0,0001002
0001003,7.0,0.0,8.0,2.0,0001003
0001004,31.0,0.0,36.0,7.0,0001004
0002011,1.0,0.0,5.0,4.0,0002011


## 4. Load Violent Crime Data to AWS

In [8]:
engine = create_engine(f"mysql://{remote_dccrime_dbuser}:{remote_dccrime_dbpwd}@{remote_db_endpoint}:{remote_db_port}/{remote_dccrime_dbname}")
conn = engine.connect()

#socioecon_data_df.to_sql('socioecon_data', conn)
crime_totals.to_sql(name='crime_totals', if_exists='replace', con=conn, chunksize=1000, index=False)